In [78]:
import pandas as pd
import numpy as np
from scipy import interpolate
from scipy.interpolate import LinearNDInterpolator, RegularGridInterpolator, interp1d

In [75]:
boom_cal = pd.read_csv('data/boom_cal.csv')
crane_load = pd.read_csv('data/crane_load.csv')

In [27]:
load = boom_cal['Unnamed: 0']
load = load.rename('cal_load(ton)').to_numpy()
load

array([ 20,  40,  60,  80, 100], dtype=int64)

In [76]:
crane_load

,work_range(m),load(ton),boom_angle(deg)
0,4.5,135.0,80.0
1,5.0,134.1,78.5
2,6.0,110.4,74.5
3,7.0,82.2,70.5
4,8.0,67.0,66.3
5,9.0,55.8,61.9
6,10.0,47.7,57.4
7,12.0,36.8,47.4
8,14.0,28.4,35.2
9,16.0,28.3,30.0


In [29]:
angle = np.array(boom_cal.columns[1:].astype(int), dtype=np.int32)
angle

array([50, 60, 70])

In [66]:
deform = boom_cal.to_numpy()[:, 1:]
deform

array([[30.5, -1. , -1. ],
       [63. , 44.2, -1. ],
       [95.5, 67.8, 45.7],
       [-1. , 91.4, 62.1],
       [-1. , -1. , 78.6]])

In [45]:
f = interpolate.interp2d(x=angle, y=load, z=deform, kind='linear')
f

C:\Users\jinbeom\AppData\Local\Temp\ipykernel_32976\2250828287.py:1: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://gist.github.com/ev-br/8544371b40f414b7eaf3fe6217209bff`

  f = interpolate.interp2d(x=angle, y=load, z=deform, kind='linear')


In [79]:
f2 = interp1d(x=crane_load['boom_angle(deg)'], y=crane_load['work_range(m)'])
f3 = interp1d(x=crane_load['work_range(m)'], y=crane_load['load(ton)'])

In [95]:
input_angle = 61.9
input_load = 55.8

In [96]:
f2(input_angle).item()

9.0

In [97]:
new_work_range = f2(input_angle).item()+(f(input_angle, input_load).item()/1000)
new_load = f3(new_work_range).item()

print(new_work_range, new_load)

9.05772331 55.332441188999994


C:\Users\jinbeom\AppData\Local\Temp\ipykernel_32976\34659970.py:1: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new code, for regular grids use `RegularGridInterpolator` instead.
        For scattered data, prefer `LinearNDInterpolator` or
        `CloughTocher2DInterpolator`.

        For more details see
        `https://gist.github.com/ev-br/8544371b40f414b7eaf3fe6217209bff`

  new_work_range = f2(input_angle).item()+(f(input_angle, input_load).item()/1000)


In [68]:
X, Y = np.meshgrid(angle, load, indexing='ij')
print(X)
print(Y)

[[50 50 50 50 50]
 [60 60 60 60 60]
 [70 70 70 70 70]]
[[ 20  40  60  80 100]
 [ 20  40  60  80 100]
 [ 20  40  60  80 100]]


In [74]:
interp = LinearNDInterpolator((angle, load), deform)

ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (3,) and arg 1 with shape (5,).